### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,39898628,79085,122834,51531,0,0,1020,5,2023-01-26 22:50,2023-01-26 22:50,1,2,2380,7,1
34,39898404,61082,79085,3,0,0,1020,6,2023-01-27 18:23,2023-01-27 18:28,1,3,3169,4,1
35,39902967,79085,12440,51531,0,0,1021,0,2023-01-28 22:00,2023-01-28 22:00,5,0,1264,4,1
36,39718560,79085,89978,1345,11,61,1021,1,2023-01-29 21:58,2023-01-29 21:58,2,2,5500,4,1
37,39902968,79085,12440,51531,0,0,1021,2,2023-01-30 22:00,2023-01-30 22:00,10,0,1028,2,1
38,39902484,90579,79085,51530,0,0,1021,3,2023-01-31 18:00,2023-01-31 18:00,1,0,2588,4,1
39,39718561,89978,79085,1345,12,61,1021,4,2023-02-01 21:58,2023-02-01 21:58,1,2,6398,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,40101,37364,2,5,1,0,0,4-3-3a,7,7,8,39898628
1,122834,29917,32654,3,7,4,3,0,Sampaio GIGANTE!,7,7,7,39898628
0,61082,36387,27904,1,3,2,0,0,tequila wăi,6,5,3,39898404
1,79085,33938,42421,1,14,1,0,0,5-4-1a,5,7,5,39898404
0,79085,16826,40641,0,30,0,0,0,4-4-2a,8,6,7,39902967
1,12440,53142,29327,0,0,3,1,0,prova 1part,4,3,4,39902967
0,79085,38356,33561,4,11,2,0,0,4-4-2a,7,7,8,39718560
1,89978,31699,36494,1,10,3,0,0,2-2,7,6,10,39718560
0,79085,14053,47026,0,45,0,0,0,4-4-2b,8,7,8,39902968
1,12440,56790,23817,0,1,1,1,0,ORTO MUSO,4,3,4,39902968


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

660

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1345

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10246,12,30,9,3,0,30,5,30525030009009,1345
1,10520,12,26,8,2,2,40,11,26529040008011,1345
2,48541,12,22,7,1,4,42,34,22508042007007,1345
3,80120,12,21,6,3,3,12,10,21502012006012,1345
4,121094,12,19,6,1,5,19,15,19504019006008,1345
5,79085,12,19,5,4,3,17,16,19501017005005,1345
6,88725,12,16,5,1,6,18,22,16496018005001,1345
7,23755,12,16,5,1,6,18,25,16493018005010,1345
8,31199,12,15,4,3,5,19,21,15498019004006,1345
9,89978,12,7,2,1,9,10,33,7477010002004,1345


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

3

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()